In [4]:
from resurfemg.data_connector.adicht_reader import AdichtReader
import pandas as pd

In [5]:
# Example Usage

emg_file_chosen = "C:/Example/Path/filename.adicht"
adicht_data = AdichtReader(emg_file_chosen)
adicht_data.print_metadata()    # Extracts and provides a tabular overview of the channels, samples, records, sampling rates, units, and time step.

selected_channel_ids = [1,7,8]
record_id = 2
resample_channels_dict={        # In this example, Channel1 has a different sampling rate compared to the other channels, so it will be resampled to match their rate
    1: 2000,                          
}

# get the sekeceted data and the sample rate of the leading channel
data_emg_df, fs_emg = adicht_data.extract_data(channel_ids= selected_channel_ids,record_id= record_id,resample_channels= resample_channels_dict)

# you can also use the resample method only to resample a single channel for specific cases
channel1_resampeld_df = adicht_data.resample_channel(channel_id= 1,record_id= 1,target_rate= 2000)     

# Select a specific time range from the DataFrame
start_time = pd.Timedelta('2 minutes 20 seconds')  # safety margin of 10seconds
end_time = pd.Timedelta('2 minutes 30 seconds')
selected_data_df = data_emg_df[(data_emg_df.index >= start_time) & (data_emg_df.index <= end_time)]

# to pass the data for the EmgDataGroup constructor, the pandas dataframe must be converted into a numpy array and transformed 
y_emg   = selected_data_df.to_numpy().T

# get the labels and units of the selected channels
label_list  = adicht_data.get_labels(selected_channel_ids)
unit_list   = adicht_data.get_units(selected_channel_ids, record_id)

FileNotFoundError: The file 'C:/Example/Path/filename.adicht' was not found.